In [96]:
import os
import glob
from typing import (
    List, 
    Dict, 
    Optional,
    Union
)
from collections import OrderedDict

In [3]:
class BIDSNameError(Exception):
    pass

class BIDSMetaDataError(Exception):
    pass

In [4]:
def is_camel_case(s: str,
                  bids_case: bool = True
                  ) -> bool:
    '''Tests if some input string is camel case (CamelCase).

    NOTE: This function is configured for BIDS use cases, in which metadata must 
        be in camel case, with the first letter being uppercase.

    Usage example:
        >>> is_camel_case("CamelCase", bids_case=True)
        True
        >>> is_camel_case("camelcase", bids_case=True)
        False
        >>> is_camel_case("camelCase", bids_case=True)
        False
        >>> is_camel_case("camelCase", bids_case=False)
        True
        
    Arguments:
        s: Input string to test.
        bids_case: In addition to being in camel case, the string's first letter must also be
            uppercase. 

    Returns:
        Boolean.
    '''
    if bids_case:
        return s != s.lower() and s != s.upper() and s[0].isupper() and "_" not in s
    else:
        return s != s.lower() and s != s.upper() and "_" not in s

In [5]:
os.getcwd()

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin'

In [10]:
# test_dir = "../tests/img.test.dir/001-001/PAR REC"
test_dir = "../tests/img.test.dir"

In [11]:
os.listdir(test_dir)

['001-001',
 '002-001',
 '003-001',
 '900XXT5',
 '901XXP5',
 '902XXY8',
 'C01-001',
 'C02-001',
 'C03-001',
 'P01-001',
 'P02-001',
 'P03-001']

In [17]:
import utils.img_dir as id
from utils.file_utils import dict_multi_update

In [16]:
dir_list,type_list = id.img_dir_list(test_dir)

In [17]:
dir_list

['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\DICOM\\20211701',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\NIFTI',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\PAR REC',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\002-001\\DICOM\\20211701',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\002-001\\NIFTI',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\002-001\\PAR REC',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\003-001\\DICOM\\20211701',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\003-001\\NIFTI',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\003-001\\PAR REC',
 'c:\\User

In [1]:
# construct BIDS dictionary
# 
# BIDS dict
# 
# subject data dict
# metadata dict (should include parameter data)
# JSON file dict
# parameter dict: 
#   - dwi: acq, dir, run [, sbref]
#   - func: task, acq, dir, ce, echo, run, rec
#       - contrast_label: bold, cbv, phase [, sbref]
#   - anat: acq, dir, ce, rec, run
#       - modality: T1w, T2w, etc.
#   - fmap: 
# 
# Relevant links:
# https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html

In [53]:
class SubInfoError(Exception):
    pass

In [54]:
class SubDataInfo():
    '''Class instance that creates a data object that organizes a subject's 
    identification (ID) number, session ID number, and the 
    path to the image data directory. This information is then stored for 
    each separate class instance, and can be accessed as shown in the example
    usage.
    
    Usage example:
        >>> sub_info = SubDataInfo(sub="002",
        ...                        data="<path/to/img/data>",
        ...                        ses="001")
        >>> sub_info.sub
        "002"
        >>> 
        >>> sub_info.ses
        "001"
    '''

    def __init__(self,
                 sub: Union[str,int],
                 data: str,
                 ses: Optional[Union[str,int]] = None):
        '''Init doc-string for the 'SubDataInfo' class. 
        
        Arguments:
            sub: Subject ID.
            data: Path to image data directory.
            ses: Session ID.
        '''
        if sub:
            self.sub: str = str(sub)
        else:
            raise SubInfoError("Subject ID was not specified")
        if data:
            self.data: str = data
        else:
            raise SubInfoError("Subject data was not specified.")
        if ses:
            self.ses: str = str(ses)
        else:
            self.ses: str = ""
    
    def __repr__(self):
        '''NOTE: Returns string represented as dictionary.'''
        return (str({"sub": self.sub,
                     "ses": self.ses,
                     "data": self.data}))

In [19]:
def construct_bids_json(meta_dict: Optional[Dict] = None,
                        json_dict: Optional[Dict] = None
                        ) -> Dict:
    '''Constructs dictionary of relevant BIDS related information that includes subject and session IDs, in addition
    to various metadata. Custom BIDS fields can also be added through the metadata dictionary.

    More information can be obtained from: https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html.

    Usage example:
        >>> bids_dict = construct_bids_dict(meta_dict,
        ...                                 json_dict)

    Arguments:
        meta_dict: Metadata dictionary that contains the relevant BIDS metadata.
        json_dict: Dictionary constructed from the BIDS JSON file.

    Returns:
        Dictionary containing BIDS related metadata.
    '''
    # BIDS informatino dictionary
    bids_info: Dict = {
        # Common metadata
        ## Scanner Hardware
        "Manufacturer":"",
        "ManufacturersModelName":"",
        "DeviceSerialNumber":"",
        "StationName":"",
        "SoftwareVersions":"",
        "HardcopyDeviceSoftwareVersion":"",
        "MagneticFieldStrength":"",
        "ReceiveCoilName":"",
        "ReceiveCoilActiveElements":"",
        "GradientSetType":"",
        "MRTransmitCoilSequence":"",
        "MatrixCoilMode":"",
        "CoilCombinationMethod":"",
        ## Sequence Specifics
        "PulseSequenceType":"",
        "ScanningSequence":"",
        "SequenceVariant":"",
        "ScanOptions":"",
        "SequenceName":"",
        "PulseSequenceDetails":"",
        "NonlinearGradientCorrection":"",
        ## In-Plane Spatial Encoding
        "NumberShots":"",
        "ParallelReductionFactorInPlane":"",
        "ParallelAcquisitionTechnique":"",
        "PartialFourier":"",
        "PartialFourierDirection":"",
        "PhaseEncodingDirection":"",
        "EffectiveEchoSpacing":"",
        "TotalReadoutTime":"",
        ## Timing Parameters
        "EchoTime":"",
        "InversionTime":"",
        "SliceTiming":"",
        "SliceEncodingDirection":"",
        "DwellTime":"",
        ## RF & Contrast
        "FlipAngle":"",
        "NegativeContrast":"",
        ## Slice Acceleration
        "MultibandAccelerationFactor":"",
        ## Anatomical landmarks
        "AnatomicalLandmarkCoordinates":"",
        ## Institution information
        "InstitutionName":"",
        "InstitutionAddress":"",
        "InstitutionalDepartmentName":"",
        # Anat
        "ContrastBolusIngredient":"",
        # Func
        "RepetitionTime":"",
        "VolumeTiming":"",
        "TaskName":"",
        "NumberOfVolumesDiscardedByScanner":"",
        "NumberOfVolumesDiscardedByUser":"",
        "DelayTime":"",
        "AcquisitionDuration":"",
        "DelayAfterTrigger":"",
        "Instructions":"",
        "TaskDescription":"",
        "CogAtlasID":"",
        "CogPOID":"",
        # Fmap
        "Units":"",
        "IntendedFor":"",
        # Custom BIDS fields
        "SourceDataFormat":""
    }

    # OrderedDict array/list
    ordered_array: List[str] = [
        "Manufacturer",
        "ManufacturersModelName",
        "DeviceSerialNumber",
        "StationName",
        "SoftwareVersions",
        "HardcopyDeviceSoftwareVersion",
        "MagneticFieldStrength",
        "ReceiveCoilName",
        "ReceiveCoilActiveElements",
        "GradientSetType",
        "MRTransmitCoilSequence",
        "MatrixCoilMode",
        "CoilCombinationMethod",
        "PulseSequenceType",
        "ScanningSequence",
        "SequenceVariant",
        "ScanOptions",
        "SequenceName",
        "PulseSequenceDetails",
        "NonlinearGradientCorrection",
        "NumberShots",
        "ParallelReductionFactorInPlane",
        "ParallelAcquisitionTechnique",
        "PartialFourier",
        "PartialFourierDirection",
        "PhaseEncodingDirection",
        "EffectiveEchoSpacing",
        "TotalReadoutTime",
        "EchoTime",
        "InversionTime",
        "SliceTiming",
        "SliceEncodingDirection",
        "DwellTime",
        "FlipAngle",
        "NegativeContrast",
        "MultibandAccelerationFactor",
        "AnatomicalLandmarkCoordinates",
        "InstitutionName",
        "InstitutionAddress",
        "InstitutionalDepartmentName",
        "ContrastBolusIngredient",
        "RepetitionTime",
        "VolumeTiming",
        "TaskName",
        "NumberOfVolumesDiscardedByScanner",
        "NumberOfVolumesDiscardedByUser",
        "DelayTime",
        "AcquisitionDuration",
        "DelayAfterTrigger",
        "Instructions",
        "TaskDescription",
        "CogAtlasID",
        "CogPOID",
        "Units",
        "IntendedFor",
        "SourceDataFormat"
    ]

    bids_dict: Dict = {}

    if meta_dict:
        pass
    else:
        meta_dict: Dict = {}
    
    if json_dict:
        pass
    else:
        json_dict = {}
    
    # Update BIDS ordered array
    meta_list: List[str] = list(meta_dict.keys())
    ordered_array.extend(meta_list)
    ordered_list: List[str] = []

    for word in ordered_array:
        # Check if the BIDS metadata field is valid
        if is_camel_case(word,bids_case=True):
            pass
        else:
            raise BIDSMetaDataError(f"Input metadata: {word} is not BIDS compliant.")
        
        # Add field to ordered array
        if word in ordered_list:
            pass
        else:
            ordered_list.append(word)
    
    # Create BIDS dictionary
    bids_dict: Dict = dict_multi_update(**bids_info,
                                        **meta_dict,
                                        **json_dict)
    
    # Create ordered BIDS dictionary
    ordered_bids_dict: OrderedDict = OrderedDict()
    for key in ordered_list:
        ordered_bids_dict[key] = bids_dict[key]
    
    return ordered_bids_dict

In [191]:
def construct_bids_name(sub_data: SubDataInfo,
                        modality_type: Optional[str] = "",
                        modality_label: Optional[str] = "",
                        acq: Optional[str] = "",
                        ce: Optional[str] = "",
                        task: Optional[str] = "",
                        acq_dir: Optional[str] = "",
                        rec: Optional[str] = "",
                        run: Optional[Union[int,str]] = "",
                        echo: Optional[Union[int,str]]  = "",
                        case_1: bool = False,
                        mag2: bool = False,
                        case_2: bool = False,
                        case_3: bool = False,
                        case_4: bool = False,
                        out_dir: Optional[str] = "",
                        zero_pad: Optional[int] = 0
                        ) -> Dict:
    '''Constructs BIDS filenames from input paraemter descriptions.

    More information can be obtained from: https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html.

    Usage example:
        >>> bids_param = construct_bids_name(sub_data=sub_data,
        ...                                  modality_type='func',
        ...                                  modality_label='bold',
        ...                                  task='rest',
        ...                                  run='01')

    Arguments:
        sub_data: SubDataInfo object that contains subject and session ID.
        modality_type: Modality type associated with the data. Valid modality types include, but are not limited to:
            * 'anat': Anatomical scans
            * 'func': Functional scans
            * 'dwi': Diffusion weighted image scans
            * 'fmap: ': Fieldmaps
            * 'swi': Susceptibility weighted image scans
        modality_label: Modality label of associated with the data. Valid modality labels include, but are not limited to:
            * 'T1w', 'T2w', 'PD', 'FLAIR', 'T1rho' (in the case of 'anat').
            * 'bold', 'cbv', 'phase' (in the case of 'func').
                * NOTE: In the case of 'func' modalities, these are referred to as "contrast_labels" in the BIDS documentation.
        acq: Acquisition description of the associated image data.
        ce: Contrast enhanced description of the associated image data.
        task: Task associated with the image data, required if 'func' is used as the modality_type.
        acq_dir: Acquisition direction of the image (e.g. PA, AP, LR, RL).
        rec: Image reconstruction description.
        run: Run number of the associated image data.
        echo: Echo number of the associated image data (, in the case of multi-echo data).
        case_1: BIDS fieldmap case 1, set to True if the following files are present:
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_phasediff.nii[.gz]
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_phasediff.json
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_magnitude1.nii[.gz]
            * [ sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_magnitude2.nii[.gz] ]
        mag2: Should be set to True, if case 1 contains a second magnitude image.
        case_2: BIDS fieldmap case 2, set to True if the following files are present:
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_phase1.nii[.gz]
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_phase1.json
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_phase2.nii[.gz]
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_phase2.json
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_magnitude1.nii[.gz]
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_magnitude2.nii[.gz]
        case_3: BIDS fieldmap case 3, set to True if the following files are present:
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_magnitude.nii[.gz]
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_fieldmap.nii[.gz]
            * sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_fieldmap.json
        case_4: BIDS fieldmap case 4, set to True if the following files are present:
            * sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>]_dir-<label>[_run-<index>]_epi.nii[.gz]
            * sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>]_dir-<label>[_run-<index>]_epi.json
        out_dir: Target output directory for a subject's BIDS files.
        zero_pad: Number of zeroes to zeropad the output value.

    Returns:
        Nested dictionary that contains the necessary information to name BIDS files.
    
    Raises:
        BIDSNameError: Error raised if:
            * 'anat' is the speicified modality_type, but no modality_label is specified.
            * 'func' is the speicified modality_type, but no task is specified.
            * 'fmap' is the speicified modality_type, but no fieldmap 'case' is specified.
    '''
    # BIDS parameter dictionary
    bids_param: Dict = {
        "info":{
            "sub":"",
            "ses":""
        },
        "anat":{
            "acq":"",
            "ce":"",
            "rec":"",
            "run":"",
            "modality_label":""
        },
        "func":{
            "task":"",
            "acq":"",
            "ce":"",
            "dir":"",
            "rec":"",
            "run":"",
            "echo":"",
            "modality_label":""
        },
        "dwi":{
            "acq":"",
            "dir":"",
            "run":"",
            "modality_label":""
        },
        "fmap":{
            "acq":"",
            "run":"",
            # Case 1: Phase difference image and at least one magnitude image
            "case1":{
                "phasediff":"",
                "magnitude1":"",
                "magnitude2":""
            },
            # Case 2: Two phase images and two magnitude images
            "case2":{
                "phase1":"",
                "phase2":"",
                "magnitude1":"",
                "magnitude2":""
            },
            # Case 3: A real fieldmap image
            "case3":{
                "magnitude":"",
                "fieldmap":""
            },
            # Case 4: Multiple phase encoded directions ("pepolar")
            "case4":{
                "ce":"",
                "dir":"",
                "modality_label": "epi"
            }
        }
    }

    # Update subject and session ID in BIDS parameter dictionary
    bids_param["info"].update({"sub":sub_data.sub,
                               "ses":sub_data.ses})

    if modality_type.lower() == 'anat':
        if not modality_label:
            raise BIDSNameError("Modality label required for anatomical data (e.g. 'T1w').")
        bids_param["anat"]["acq"] = acq
        bids_param["anat"]["ce"] = ce
        bids_param["anat"]["rec"] = rec
        bids_param["anat"]["run"] = run
        bids_param["anat"]["modality_label"] = modality_label
    elif modality_type.lower() == 'func':
        if not task:
            raise BIDSNameError("Task label required for functional data.")
        bids_param["func"]["task"] = task
        bids_param["func"]["acq"] = acq
        bids_param["func"]["ce"] = ce
        bids_param["func"]["dir"] = acq_dir
        bids_param["func"]["rec"] = rec
        bids_param["func"]["run"] = run
        bids_param["func"]["echo"] = echo
        bids_param["func"]["modality_label"] = modality_label
    elif modality_type.lower() == 'dwi':
        bids_param["dwi"]["acq"] = acq
        bids_param["dwi"]["dir"] = acq_dir
        bids_param["dwi"]["run"] = run
        bids_param["dwi"]["modality_label"] = modality_label
    elif modality_type.lower() == 'fmap':
        bids_param["fmap"]["acq"] = acq
        bids_param["fmap"]["run"] = run
        if case_1:
            bids_param["fmap"]["case1"]["phasediff"] = "phasediff"
            bids_param["fmap"]["case1"]["magnitude1"] = "magnitude1"
            if mag2:
                bids_param["fmap"]["case1"]["magnitude2"] = "magnitude2"
        elif case_2:
            bids_param["fmap"]["case2"]["phase1"] = "phase1"
            bids_param["fmap"]["case2"]["phase2"] = "phase2"
            bids_param["fmap"]["case2"]["magnitude1"] = "magnitude1"
            bids_param["fmap"]["case2"]["magnitude2"] = "magnitude2"
        elif case_3:
            bids_param["fmap"]["case3"]["magnitude"] = "magnitude"
            bids_param["fmap"]["case3"]["fieldmap"] = "fieldmap"
        elif case_4:
            bids_param["func"]["ce"] = ce
            bids_param["func"]["dir"] = acq_dir
        else:
            raise BIDSNameError("Fieldmap data was specified, however, no BIDS fieldmap case was specified.")
    elif modality_type:
        bids_param.update({modality_label:{"acq":acq,
                                           "ce":ce,
                                           "dir":acq_dir,
                                           "rec":rec,
                                           "run":run,
                                           "echo":echo}})
    elif modality_type == "":
        bids_param.update({"unknown":{"task":task,
                                      "acq":acq,
                                      "ce":ce,
                                      "dir":acq_dir,
                                      "rec":rec,
                                      "run":run,
                                      "echo":echo,
                                      "modality_label":modality_label}})

    # Obtain run number if not specified
    if run is None or run == "":
        if out_dir:
            run = num_runs(directory=out_dir,
                        modality_type=modality_type,
                        modality_label=modality_label,
                        bids_dict=bids_param,
                        zero_pad=zero_pad)
        else:
            run = zeropad(num=1,num_zeros=zero_pad)
        
        # Back track to fill in run number
        bids_param = construct_bids_name(sub_data=sub_data,
                                        modality_type=modality_type,
                                        modality_label=modality_label,
                                        acq=acq,
                                        ce=ce,
                                        task=task,
                                        acq_dir=acq_dir,
                                        rec=rec,
                                        run=run,
                                        echo=echo,
                                        case_1=case_1,
                                        mag2=mag2,
                                        case_2=case_2,
                                        case_3=case_3,
                                        case_4=case_4)

    return bids_param

In [12]:
# BIDS information dictionary
bids_info = {
    # Common metadata
    ## Scanner Hardware
    "Manufacturer":"",
    "ManufacturersModelName":"",
    "DeviceSerialNumber":"",
    "StationName":"",
    "SoftwareVersions":"",
    "HardcopyDeviceSoftwareVersion":"",
    "MagneticFieldStrength":"",
    "ReceiveCoilName":"",
    "ReceiveCoilActiveElements":"",
    "GradientSetType":"",
    "MRTransmitCoilSequence":"",
    "MatrixCoilMode":"",
    "CoilCombinationMethod":"",
    ## Sequence Specifics
    "PulseSequenceType":"",
    "ScanningSequence":"",
    "SequenceVariant":"",
    "ScanOptions":"",
    "SequenceName":"",
    "PulseSequenceDetails":"",
    "NonlinearGradientCorrection":"",
    ## In-Plane Spatial Encoding
    "NumberShots":"",
    "ParallelReductionFactorInPlane":"",
    "ParallelAcquisitionTechnique":"",
    "PartialFourier":"",
    "PartialFourierDirection":"",
    "PhaseEncodingDirection":"",
    "EffectiveEchoSpacing":"",
    "TotalReadoutTime":"",
    ## Timing Parameters
    "EchoTime":"",
    "InversionTime":"",
    "SliceTiming":"",
    "SliceEncodingDirection":"",
    "DwellTime":"",
    ## RF & Contrast
    "FlipAngle":"",
    "NegativeContrast":"",
    ## Slice Acceleration
    "MultibandAccelerationFactor":"",
    ## Anatomical landmarks
    "AnatomicalLandmarkCoordinates":"",
    ## Institution information
    "InstitutionName":"",
    "InstitutionAddress":"",
    "InstitutionalDepartmentName":"",
    # Anat
    "ContrastBolusIngredient":"",
    # Func
    "RepetitionTime":"",
    "VolumeTiming":"",
    "TaskName":"",
    "NumberOfVolumesDiscardedByScanner":"",
    "NumberOfVolumesDiscardedByUser":"",
    "DelayTime":"",
    "AcquisitionDuration":"",
    "DelayAfterTrigger":"",
    "Instructions":"",
    "TaskDescription":"",
    "CogAtlasID":"",
    "CogPOID":"",
    # Fmap
    "Units":"",
    "IntendedFor":"",
    # Custom BIDS fields
    "SourceDataFormat":""
}

In [13]:
# list(bids_info.keys())

In [14]:
# * Add parameter specific metadata to BIDS info dictionary *
# BIDS parameter dictionary
bids_param = {
    "info":{
        "sub":"",
        "ses":""
    },
    "anat":{
        "acq":"",
        "ce":"",
        "rec":"",
        "run":"",
        "modality_label":""
    },
    "func":{
        "task":"",
        "acq":"",
        "ce":"",
        "dir":"",
        "rec":"",
        "run":"",
        "echo":"",
        "modality_label":""
    },
    "dwi":{
        "acq":"",
        "dir":"",
        "run":"",
        "modality_label":""
    },
    "fmap":{
        "acq":"",
        "run":"",
        # Case 1: Phase difference image and at least one magnitude image
        "case1":{
            "phasediff":"",
            "magnitude1":"",
            "magnitude2":""
        },
        # Case 2: Two phase images and two magnitude images
        "case2":{
            "phase1":"",
            "phase2":"",
            "magnitude1":"",
            "magnitude2":""
        },
        # Case 3: A real fieldmap image
        "case3":{
            "magnitude":"",
            "fieldmap":""
        },
        # Case 4: Multiple phase encoded directions ("pepolar")
        "case4":{
            "ce":"",
            "dir":""
        }
    }
}

In [15]:
bids_param['info']

{'sub': '', 'ses': ''}

In [16]:
bids_param["info"].update({"sub":"001","ses":"001"})
bids_param["info"]

{'sub': '001', 'ses': '001'}

In [17]:
list(bids_param.keys())

['info', 'anat', 'func', 'dwi', 'fmap']

In [19]:
bids_param["fmap"]["case1"]

{'phasediff': '', 'magnitude1': '', 'magnitude2': ''}

#### Test case for `OrderedDict`

In [32]:
mydict = {'Rust': {'definition':'rusts definition'},
          'Iron': {'definition':'iron definition'},
          'Pyrite': {'definition':'pyrite definition'}}

myorder = ['Pyrite', 'Rust', 'Iron']

In [34]:
ordered = OrderedDict()
for k in myorder:
    ordered[k] = mydict[k]
ordered

OrderedDict([('Pyrite', {'definition': 'pyrite definition'}),
             ('Rust', {'definition': 'rusts definition'}),
             ('Iron', {'definition': 'iron definition'})])

In [44]:
len(construct_bids_json())

56

In [1]:
t = [ 't' , 'g', 'l']

In [7]:
s = []

In [8]:
t.append("t")

In [9]:
for word in t:
    if word not in s:
        s.append(word)
s

['t', 'g', 'l']

In [49]:
# def is_camel_case(s: str) -> bool:
#     '''Tests if some input string is camel case (CamelCase).

#     Usage example:
#         >>> is_camel_case("CamelCase")
#         True
#         >>>
#         >>> is_camel_case("camelCase")
#         False
#         >>> is_camel_case("camelcase")
#         False
        
#     Arguments:
#         s: Input string to test.

#     Returns:
#         Boolean.
#     '''
#     return s != s.lower() and s != s.upper() and s[0].isupper() and "_" not in s

In [53]:
is_camel_case("corCpm")

False

### Test 

In [21]:
construct_bids_json()

OrderedDict([('Manufacturer', ''),
             ('ManufacturersModelName', ''),
             ('DeviceSerialNumber', ''),
             ('StationName', ''),
             ('SoftwareVersions', ''),
             ('HardcopyDeviceSoftwareVersion', ''),
             ('MagneticFieldStrength', ''),
             ('ReceiveCoilName', ''),
             ('ReceiveCoilActiveElements', ''),
             ('GradientSetType', ''),
             ('MRTransmitCoilSequence', ''),
             ('MatrixCoilMode', ''),
             ('CoilCombinationMethod', ''),
             ('PulseSequenceType', ''),
             ('ScanningSequence', ''),
             ('SequenceVariant', ''),
             ('ScanOptions', ''),
             ('SequenceName', ''),
             ('PulseSequenceDetails', ''),
             ('NonlinearGradientCorrection', ''),
             ('NumberShots', ''),
             ('ParallelReductionFactorInPlane', ''),
             ('ParallelAcquisitionTechnique', ''),
             ('PartialFourier', ''),
      

In [187]:
s = SubDataInfo(sub="001",ses=None,data=os.getcwd())

In [188]:
s.sub

'001'

In [190]:
construct_bids_name(s,modality_type='swi',modality_label='swi',zero_pad=2)

<class 'int'>
<class 'int'>


{'info': {'sub': '001', 'ses': ''},
 'anat': {'acq': '', 'ce': '', 'rec': '', 'run': '', 'modality_label': ''},
 'func': {'task': '',
  'acq': '',
  'ce': '',
  'dir': '',
  'rec': '',
  'run': '',
  'echo': '',
  'modality_label': ''},
 'dwi': {'acq': '', 'dir': '', 'run': '', 'modality_label': ''},
 'fmap': {'acq': '',
  'run': '',
  'case1': {'phasediff': '', 'magnitude1': '', 'magnitude2': ''},
  'case2': {'phase1': '', 'phase2': '', 'magnitude1': '', 'magnitude2': ''},
  'case3': {'magnitude': '', 'fieldmap': ''},
  'case4': {'ce': '', 'dir': '', 'modality_label': 'epi'}},
 'swi': {'acq': '', 'ce': '', 'dir': '', 'rec': '', 'run': '01', 'echo': ''}}

In [178]:
def num_runs(directory: Optional[str] = "",
             modality_type: Optional[str] = "",
             modality_label: Optional[str] = "",
             bids_dict: Optional[Dict] = None,
             zero_pad: Optional[int] = None
             ) -> Union[int,str]:
    '''Counts the number of similarly named files in a directory to obtain a unique run number.
    Optimal use of this function requires the:
        * search directory (directory)
        * modality type (modality_type)
        * BIDS filename dictionary (bids_dict)
    
    NOTE: The optional input bids_dict is a nested dictionary constructed by the function `construct_bids_name`.
    
    Usage example:
        >>> num_runs(directory: str,
        ...          modality_type='func',
        ...          modality_label='bold,
        ...          bids_dict=bids_dict,
        ...          zero_pad=3)
        '001'

    Arguments:
        directory: Input directory to search.
        modality_type: Modality type associated with the data. Valid modality types include, but are not limited to:
            * 'anat': Anatomical scans
            * 'func': Functional scans
            * 'dwi': Diffusion weighted image scans
            * 'fmap: ': Fieldmaps
            * 'swi': Susceptibility weighted image scans
        modality_label: Modality label of associated with the data. Valid modality labels include, but are not limited to:
            * 'T1w', 'T2w', 'PD', 'FLAIR', 'T1rho' (in the case of 'anat').
            * 'bold', 'cbv', 'phase' (in the case of 'func').
                * NOTE: In the case of 'func' modalities, these are referred to as "contrast_labels" in the BIDS documentation.
        bids_dict: Nested BIDS name dictionary that contains relevant information to naming files.
        zero_pad: Number of zeroes to zeropad the output value.

    Returns:
        Integer (or string should the value be zeropadded).
    '''
    if os.path.exists(directory):
        directory: str = os.path.abspath(directory)
    elif zero_pad:
        return zeropad(num=1,num_zeros=zero_pad)
    else:
        return 1
    
    if modality_type and modality_label and not bids_dict:
        glob_str: str = os.path.join(directory,f"*{modality_label}*{modality_type}*.nii*")
        runs: str = os.path.join(directory,f"*{glob_str}.nii*")
        num: int = len(glob.glob(runs)) + 1

        if zero_pad:
            return zeropad(num=num,num_zeros=zero_pad)
        else:
            return num
    
    if bids_dict:
        pass
    elif zero_pad:
        return zeropad(num=1,num_zeros=zero_pad)
    else:
        return 1
    
    # Construct glob string
    tmp_list: List[str] = []
    for item in list(bids_dict[modality_type].keys()):
        if bids_dict[modality_type][item] == "":
            pass
        else:
            tmp_list.append(bids_dict[modality_type][item])
            tmp_list.append("*")
    glob_str: str = ''.join(tmp_list)
    runs: str = os.path.join(directory,f"*{glob_str}.nii*")
    num: int = len(glob.glob(runs)) + 1

    if zero_pad:
        return zeropad(num=num,num_zeros=zero_pad)
    else:
        return num

In [66]:
d = construct_bids_name(s,modality_type='func',modality_label='bold',task='rest'); d

{'info': {'sub': '001', 'ses': ''},
 'anat': {'acq': '', 'ce': '', 'rec': '', 'run': '', 'modality_label': ''},
 'func': {'task': 'rest',
  'acq': '',
  'ce': '',
  'dir': '',
  'rec': '',
  'run': '',
  'echo': '',
  'modality_label': 'bold'},
 'dwi': {'acq': '', 'dir': '', 'run': '', 'modality_label': ''},
 'fmap': {'acq': '',
  'run': '',
  'case1': {'phasediff': '', 'magnitude1': '', 'magnitude2': ''},
  'case2': {'phase1': '', 'phase2': '', 'magnitude1': '', 'magnitude2': ''},
  'case3': {'magnitude': '', 'fieldmap': ''},
  'case4': {'ce': '', 'dir': '', 'modality_label': 'epi'}}}

In [70]:
m = 'func'

In [71]:
d[m]

{'task': 'rest',
 'acq': '',
 'ce': '',
 'dir': '',
 'rec': '',
 'run': '',
 'echo': '',
 'modality_label': 'bold'}

In [73]:
list(d[m].keys())

['task', 'acq', 'ce', 'dir', 'rec', 'run', 'echo', 'modality_label']

In [95]:
tmp_list = []
for item in list(d[m].keys()):
    if d[m][item] == "":
        pass
    else:
        tmp_list.append(d[m][item])
        tmp_list.append("*")
tmp_list
''.join(tmp_list)

'rest*bold*'

In [97]:
glob_str = ''.join(tmp_list); glob_str

'rest*bold*'

In [115]:
# d

In [160]:
# num_runs(os.getcwd(),"func","bold",d)
num_runs()

1

In [119]:
n = 'j'

In [120]:
n.zfill(3)

'00j'

In [121]:
int("j")

ValueError: invalid literal for int() with base 10: 'j'

In [181]:
def zeropad(num: Union[str,int],
            num_zeros: int = 2
            ) -> str:
    '''Zeropads a number, should that number be an int or str.
    
    Usage example:
        >>> zeropad(5,2)
        '05'
        >>> zeropad('5',2)
        '05'

    Arguments:
        num: Input number (as str or int) to zeropad.
        num_zeros: Number of zeroes to pad with.

    Returns:
        Zeropadded string of the number, or the original string if the input string could not
            be represented as an integer.

    Raises:
        TypeError: Error that arises if floats are passed as an argument.
    '''
    if type(num) is float:
        raise TypeError("Only integers and strings can be used with the zeropad function.")
    try:
        num: str = str(num)
        num: str = num.zfill(num_zeros)
        return num
    except ValueError:
        return num

In [177]:
zeropad(2,1)

'2'

In [155]:
"1" + "2"

'12'